In [8]:
import torch

In [9]:
import torch
print(torch.cuda.is_available())


True


In [10]:
import pandas as pd

In [24]:
train_dataset=pd.read_csv("/content/fashion-mnist_train.csv")

In [26]:
train_dataset.shape

(33588, 785)

In [27]:
train_features=train_dataset.drop(columns=["label"]).values
train_labels=train_dataset["label"].values

In [28]:
test_dataset=pd.read_csv("/content/fashion-mnist_test.csv")

In [29]:
test_features=test_dataset.drop(columns=["label"]).values
test_labels=test_dataset["label"].values

In [30]:
test_dataset.shape

(10000, 785)

In [31]:
from torch.utils.data import Dataset,DataLoader

In [32]:
class customdataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [33]:
train_datasetss=customdataset(train_features,train_labels)
test_datasetss=customdataset(test_features,test_labels)

In [34]:
len(train_datasetss)

33588

In [35]:
train_datasetss[0]

(tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   4.,   0.,   0.,   0.,   0.,   0.,  62.,  61.,
          21.,  29.,  23.,  51., 136.,  61.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  88.,
         201., 228., 225., 255., 115.,  62., 137., 255., 235., 222., 255., 135.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,  47., 252., 2

In [36]:
dataloader=DataLoader(train_datasetss,batch_size=32,shuffle=True)

In [37]:
len(dataloader)

1050

In [38]:
import torch.nn as nn

In [43]:
## create model
class models(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.network=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,features):
    out=self.network(features)
    return out

In [41]:
epochs=100
learning_rate=0.01

In [44]:
model=models(train_features.shape[1])

In [45]:
## move to gpu first step is
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [46]:
model=model.to(device)

In [47]:
## intialize loss function
criterion=nn.CrossEntropyLoss()

In [51]:
## OPtmizers
import torch.optim as optim

optmizers=optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features,batch_labels in dataloader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    output=model(batch_features)
    loss=criterion(output,batch_labels)
    optmizers.zero_grad()
    loss.backward()
    optmizers.step()
    total_epoch_loss+=loss.item()
  avg_epoch_loss=total_epoch_loss\len(train_dataloader)